In [15]:
import os
import pandas as pd
import win32com.client
from datetime import datetime


def processar_aniversario_empresa():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversario - Empresa.xlsx"
    NOME_ABA = "Sheet1"

    MES_PROCESSAR = 8  # Agosto
    ANO_ATUAL = datetime.now().year

    PASTA_CARDS = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes - Março"
    CAMINHO_MARCA = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Marca_GovTi.png"
    EMAIL_DESTINO = "carla.vieira@grendene.com.br"

    MESES_ABREV = {
        1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
        7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
    }

    try:
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA)

        # Transformar mês em número, se estiver como texto
        if df['aniversario_empresa MÊS'].dtype == 'object':
            meses_map = {v: k for k, v in MESES_ABREV.items()}
            df['aniversario_empresa MÊS'] = (
                df['aniversario_empresa MÊS']
                .str.strip()
                .str.capitalize()
                .map(meses_map)
            )

        # Filtrar apenas para o mês que queremos processar
        df_mes = df[df['aniversario_empresa MÊS'] == MES_PROCESSAR].copy()

        if df_mes.empty:
            print(f"[INFO] Não há aniversariantes no mês {MESES_ABREV[MES_PROCESSAR]}.")
            return

        # Remover linhas onde o dia está vazio
        df_mes = df_mes[df_mes['aniversario_empresa DIA'].notna()]

        # Verifica se tem aniversariantes válidos
        if df_mes.empty:
            print(f"[INFO] Nenhum aniversariante válido no mês {MESES_ABREV[MES_PROCESSAR]}.")
            return

        arquivos_cartoes = os.listdir(PASTA_CARDS) if os.path.exists(PASTA_CARDS) else []
        outlook = win32com.client.Dispatch("Outlook.Application")

        # Agrupar por dia
        for dia, grupo in df_mes.groupby('aniversario_empresa DIA'):
            data_evento = datetime(ANO_ATUAL, MES_PROCESSAR, int(dia), 8, 0)

            nomes = grupo['Nome Funcionário'].tolist()
            tempos = grupo['Tempo Empresa'].fillna(0).astype(int).tolist()

            def formatar_tempo(t):
                return f"{t} ano" if t == 1 else f"{t} anos"

            nomes_txt = "\n".join(
                f"{nome}, {formatar_tempo(tempo)};" for nome, tempo in zip(nomes, tempos)
            )
            nomes_html = "<br>".join(
                f"<b>{nome}</b>, {formatar_tempo(tempo)};" for nome, tempo in zip(nomes, tempos)
            )

            print(f"\n[PROCESSANDO] {dia:02d}/{MES_PROCESSAR}: {nomes_txt}")

            # === Criar Evento ===
            appointment = outlook.CreateItem(1)  # olAppointmentItem

            if len(nomes) == 1:
                titulo = f"Aniversário de Empresa: {nomes[0]}"
            else:
                titulo = f"Aniversário de Empresa dos Colaboradores - {dia:02d}/{MES_PROCESSAR}"

            appointment.Subject = titulo
            appointment.Start = data_evento
            appointment.Duration = 30
            appointment.AllDayEvent = False
            appointment.ReminderSet = True
            appointment.ReminderMinutesBeforeStart = 1440
            appointment.BusyStatus = 0

            corpo_evento = f"""Olá!
Segue lembrete de aniversário de empresa dos colaboradores:
{nomes_txt}

ATENÇÃO!: No dia do aniversário de empresa dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.

Atenciosamente,
"""
            appointment.Body = corpo_evento

            # Anexar cartões no evento
            anexados = 0
            for nome in nomes:
                nome_lower = nome.lower()
                encontrado = False
                for arquivo in arquivos_cartoes:
                    if nome_lower in arquivo.lower():
                        caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                        appointment.Attachments.Add(caminho_cartao)
                        anexados += 1
                        encontrado = True
                        break
                if not encontrado:
                    print(f"[AVISO] Cartão não encontrado para {nome} (evento)")

            print(f"[INFO] {anexados} cartões anexados ao evento.")
            appointment.Display()  # Mostrar antes de enviar

            # === Criar Email ===
            mail = outlook.CreateItem(0)  # olMailItem
            mail.Subject = f"Lembrete de Aniversário de Empresa - {dia:02d}/{MES_PROCESSAR}/{ANO_ATUAL}"
            mail.To = EMAIL_DESTINO

            # Anexar cartões no email
            anexados_email = 0
            for nome in nomes:
                nome_lower = nome.lower()
                encontrado = False
                for arquivo in arquivos_cartoes:
                    if nome_lower in arquivo.lower():
                        caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                        mail.Attachments.Add(caminho_cartao)
                        anexados_email += 1
                        encontrado = True
                        break
                if not encontrado:
                    print(f"[AVISO] Cartão não encontrado para {nome} (email)")

            # Adicionar marca inline (somente no email)
            img_tag = ""
            if os.path.exists(CAMINHO_MARCA):
                attachment = mail.Attachments.Add(CAMINHO_MARCA)
                attachment.PropertyAccessor.SetProperty(
                    "http://schemas.microsoft.com/mapi/proptag/0x3712001F", "marca"
                )
                img_tag = '<p><img src="cid:marca" style="width:300px;"></p>'

            corpo_email = f"""
<html>
<body>
<p>Olá!</p>

<p>Segue lembrete de aniversário de empresa dos colaboradores:<br>
{nomes_html}</p>

<p><b>ATENÇÃO!:</b> No dia do aniversário de empresa dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.</p>

<p>Atenciosamente,</p>

{img_tag}

</body>
</html>
"""
            mail.HTMLBody = corpo_email

            print(f"[INFO] {anexados_email} cartões anexados ao email.")
            mail.Display()  # Mostrar antes de enviar

            print(f"[OK] Evento e email preparados para o dia {dia:02d}/{MES_PROCESSAR}")

        print("\n[FINALIZADO] Processamento concluído!")

    except Exception as e:
        print(f"[ERRO] Falha geral na automação: {str(e)}")


if __name__ == "__main__":
    processar_aniversario_empresa()



[PROCESSANDO] 01/8: DOUGLAS AGUIAR ALCANTARA, 1 ano;
MILENA STEMPKOSKI, 2 anos;
LAURA NALLEM FRANCISQUETTI, 2 anos;
ANTONIO HELDER SANTIAGO MOREIRA, 2 anos;
[AVISO] Cartão não encontrado para DOUGLAS AGUIAR ALCANTARA (evento)
[AVISO] Cartão não encontrado para MILENA STEMPKOSKI (evento)
[AVISO] Cartão não encontrado para LAURA NALLEM FRANCISQUETTI (evento)
[AVISO] Cartão não encontrado para ANTONIO HELDER SANTIAGO MOREIRA (evento)
[INFO] 0 cartões anexados ao evento.
[AVISO] Cartão não encontrado para DOUGLAS AGUIAR ALCANTARA (email)
[AVISO] Cartão não encontrado para MILENA STEMPKOSKI (email)
[AVISO] Cartão não encontrado para LAURA NALLEM FRANCISQUETTI (email)
[AVISO] Cartão não encontrado para ANTONIO HELDER SANTIAGO MOREIRA (email)
[INFO] 0 cartões anexados ao email.
[OK] Evento e email preparados para o dia 01/8

[PROCESSANDO] 02/8: GIRLEI DIAS, 4 anos;
[AVISO] Cartão não encontrado para GIRLEI DIAS (evento)
[INFO] 0 cartões anexados ao evento.
[AVISO] Cartão não encontrado para 

In [9]:
## Codigo Teste
import os
import pandas as pd
import win32com.client
from datetime import datetime


def processar_aniversario_empresa():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariante - Empresa.xlsx"
    NOME_ABA = "Sheet1"

    MES_CARDS = 3  # Março (pasta dos cartões)
    MES_PROCESSAR = 8  # Agosto (mês do evento)
    ANO = datetime.now().year

    MESES_ABREV = {
        1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
        7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
    }

    PASTA_CARDS = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes - Março"
    CAMINHO_MARCA = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Marca_GovTi.png"
    EMAIL_DESTINO = "carla.vieira@grendene.com.br"

    try:
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA)

        mes_abrev = MESES_ABREV[MES_PROCESSAR]
        df_mes = df[df['aniversario_empresa MÊS'].str.strip().str.capitalize() == mes_abrev]

        if df_mes.empty:
            print(f"[INFO] Não há aniversariantes de empresa no mês {mes_abrev}")
            return

        nomes = df_mes['Nome Funcionário'].tolist()
        tempos = df_mes['Tempo Empresa'].tolist()

        if not nomes:
            print("[INFO] Nenhum colaborador encontrado para o mês.")
            return

        outlook = win32com.client.Dispatch("Outlook.Application")
        arquivos_cartoes = os.listdir(PASTA_CARDS) if os.path.exists(PASTA_CARDS) else []

        # Montagem dos nomes formatados
        def formatar_tempo(t):
            return f"{t} ano" if t == 1 else f"{t} anos"

        nomes_txt = "\n".join(f"{nome}, {formatar_tempo(tempo)};" for nome, tempo in zip(nomes, tempos))
        nomes_html = "<br>".join(f"<b>{nome}</b>, {formatar_tempo(tempo)};" for nome, tempo in zip(nomes, tempos))

        data_evento = datetime(ANO, MES_PROCESSAR, 1, 8, 0)

        print(f"\n[PROCESSANDO] Aniversário de empresa para {nomes_txt}")

        # === Criar Evento (sem imagem da marca) ===
        appointment = outlook.CreateItem(1)

        if len(nomes) == 1:
            titulo = f"Aniversário de Empresa: {nomes[0]}"
        else:
            titulo = f"Aniversário de Empresa dos Colaboradores"

        appointment.Subject = titulo
        appointment.Start = data_evento
        appointment.Duration = 30
        appointment.AllDayEvent = False
        appointment.ReminderSet = True
        appointment.ReminderMinutesBeforeStart = 1440
        appointment.BusyStatus = 0

        corpo_evento = f"""Olá!
Segue lembrete de aniversário de empresa para os colaboradores:
{nomes_txt}

ATENÇÃO!: No dia do aniversário de empresa dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.

Atenciosamente,
"""
        appointment.Body = corpo_evento

        # Anexar cartões no evento
        anexados = 0
        for nome in nomes:
            nome_lower = nome.lower()
            for arquivo in arquivos_cartoes:
                if nome_lower in arquivo.lower():
                    caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                    appointment.Attachments.Add(caminho_cartao)
                    anexados += 1
                    break
            else:
                print(f"[AVISO] Cartão não encontrado para {nome} (evento)")

        print(f"[INFO] {anexados} cartões anexados ao evento.")

        appointment.Display()  # mostra para revisão

        # === Criar Email (com imagem da marca inline) ===
        mail = outlook.CreateItem(0)
        mail.Subject = f"Lembrete de Aniversário de Empresa - {data_evento.strftime('%d/%m/%Y')}"
        mail.To = EMAIL_DESTINO

        # Anexar cartões no email
        anexados_email = 0
        for nome in nomes:
            nome_lower = nome.lower()
            for arquivo in arquivos_cartoes:
                if nome_lower in arquivo.lower():
                    caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                    mail.Attachments.Add(caminho_cartao)
                    anexados_email += 1
                    break
            else:
                print(f"[AVISO] Cartão não encontrado para {nome} (email)")

        # Adicionar imagem da marca inline (somente no e-mail)
        img_tag = ""
        if os.path.exists(CAMINHO_MARCA):
            attachment = mail.Attachments.Add(CAMINHO_MARCA)
            attachment.PropertyAccessor.SetProperty(
                "http://schemas.microsoft.com/mapi/proptag/0x3712001F", "marca"
            )
            img_tag = '<p><img src="cid:marca" style="width:300px;"></p>'

        corpo_email = f"""
<html>
<body>
<p>Olá!</p>

<p>Segue lembrete de aniversário de empresa para os colaboradores:<br>
{nomes_html}</p>

<p><b>ATENÇÃO!:</b> No dia do aniversário de empresa dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.</p>

<p>Atenciosamente,</p>

{img_tag}

</body>
</html>
"""
        mail.HTMLBody = corpo_email

        print(f"[INFO] {anexados_email} cartões anexados ao email.")

        mail.Display()  # mostra para revisão

        print(f"[OK] Evento e email preparados para {nomes_txt}")
        print("\n[FINALIZADO] Processamento concluído!")

    except Exception as e:
        print(f"[ERRO] Falha geral na automação: {str(e)}")


if __name__ == "__main__":
    processar_aniversario_empresa()



[PROCESSANDO] Aniversário de empresa para NAIARA DA SILVA CAMARGO, 1 ano;
DENER DAVI SOUSA LIMA, 3 anos;
MILENA STEMPKOSKI, 2 anos;
CAMILA DE AZEVEDO OLIVEIRA, 2 anos;
FRANCISCO DOUGLAS VASCONCELOS LOPES, 1 ano;
FRANCISCO DOUGLAS VASCONCELOS LOPES, 1 ano;
GIRLEI DIAS, 4 anos;
JOSE ANDERSON PONTE PRADO, 3 anos;
PRISCILA MERZONI, 4 anos;
DOUGLAS AGUIAR ALCANTARA, 1 ano;
MARCIA KESSLER KARSBURG, 1 ano;
LAURA NALLEM FRANCISQUETTI, 2 anos;
LAURO CESAR BRAGA CHAVES, 12 anos;
ALBERTO DO NASCIMENTO FERREIRA FILHO, 3 anos;
TAILOR PIUCO, 5 anos;
DANIEL RODRIGUES DE PAULA, 16 anos;
LUCAS WEBBER MOLIN, 1 ano;
CLEBER BALBINOT, 3 anos;
ANTONIO HELDER SANTIAGO MOREIRA, 2 anos;
ANDRE FELIPE GREHS, 1 ano;
BRUNO DE SOUZA REINALDO, 1 ano;
VINICIUS FAGUNDES CARDOSO, 2 anos;
[AVISO] Cartão não encontrado para NAIARA DA SILVA CAMARGO (evento)
[AVISO] Cartão não encontrado para MILENA STEMPKOSKI (evento)
[AVISO] Cartão não encontrado para FRANCISCO DOUGLAS VASCONCELOS LOPES (evento)
[AVISO] Cartão não encon